In [36]:
#MINIST
import torch
from mmcls.apis import init_model, show_result_pyplot
import mmcv

mnist_config = "config/MINST_LeNet5.py"
imagenet_config = "config/ImageNet_ResNet50.py"
checkpoint_file = "/home/akiyo/nfs/zhang/pretrained_weights/resnet50_8xb256-rsb-a1-600e_in1k_20211228-20e21305.pth"
checkpoint_file = None

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = init_model(mnist_config, checkpoint_file, device=device)

In [37]:
#查看模型继承结构/The model's inheritance relationship
model.__class__.__mro__

(mmcls.models.classifiers.image.ImageClassifier,
 mmcls.models.classifiers.base.BaseClassifier,
 mmcv.runner.base_module.BaseModule,
 torch.nn.modules.module.Module,
 object)

In [53]:
import os
import time
from mmcv import Config
from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

cfg = model.cfg

# Build the classifier
#model.init_weights()

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES

# Only support single GPU
train_model(
    model, datasets, cfg, distributed=False, validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()), meta=dict()
)


2022-12-10 19:00:55,715 - mmcls - INFO - load checkpoint from local path: /home/akiyo/sandbox/work_dirs/MINST/latest.pth
2022-12-10 19:00:55,721 - mmcls - INFO - Start running, host: akiyo@vali-docker, work_dir: /home/akiyo/sandbox/work_dirs/MINST
2022-12-10 19:00:55,723 - mmcls - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -----

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 1155.0 task/s, elapsed: 9s, ETA:     0s

2022-12-10 19:01:39,846 - mmcls - INFO - Epoch(val) [5][79]	accuracy_top-1: 99.1200


In [57]:
from mmcls.apis import train_model, inference_model

minist_image = "/home/akiyo/nfs/zhang/dataset/image/10022.png"
resnet_image = "/home/akiyo/nfs/zhang/dataset/image/bear.jpg"

model = init_model(mnist_config, checkpoint_file, device=device)
checkpoint_file = model.cfg.load_from

image = mmcv.imread(resnet_image, channel_order='brg')
image = mmcv.imrescale(image, 0.25)

result = inference_model(model, image)
show_result_pyplot(model, image, result)

load checkpoint from local path: /home/akiyo/sandbox/work_dirs/MINST/latest.pth
ImageClassifier(
  (backbone): LeNet5(
    (features): Sequential(
      (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
      (1): Tanh()
      (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
      (4): Tanh()
      (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
      (7): Tanh()
    )
    (classifier): Sequential(
      (0): Linear(in_features=120, out_features=84, bias=True)
      (1): Tanh()
      (2): Linear(in_features=84, out_features=10, bias=True)
    )
  )
  (head): ClsHead(
    (compute_loss): CrossEntropyLoss()
    (compute_accuracy): Accuracy()
  )
)


RuntimeError: Given groups=1, weight of size [6, 1, 5, 5], expected input[1, 3, 32, 32] to have 1 channels, but got 3 channels instead